In [264]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile
import pickle

import numpy as np
import cPickle
import random
import math
import h5py
from multiprocessing import cpu_count
import scipy.io as sio
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from gensim.models import Word2Vec, KeyedVectors
from six import iteritems
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle as skshuffle
from numpy import genfromtxt
from sklearn.metrics import accuracy_score

from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec
from gensim.models.word2vec import Vocab

import word2vec

np.random.seed(1)
random.seed(1)

# Evaluation

In [265]:
def sparse2graph(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k,v in iteritems(G)}

class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

np.random.seed(1)
random.seed(1)
def scoring(emb_filename, matfile):
    # 0. Files
    embeddings_file = emb_filename

    # 1. Load Embeddings
    embeddings = np.loadtxt(embeddings_file)
    
    ## for original deepwalk
    #model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)
    
    ## for external word2vec lib
#     model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

    # 2. Load labels
    mat = sio.loadmat(matfile)
    A = mat['network']
    graph = sparse2graph(A)
    labels_matrix = mat['group']
    labels_count = labels_matrix.shape[1]
    mlb = MultiLabelBinarizer(range(labels_count))

    # Map nodes to their features (note:  assumes nodes are labeled as integers 1:N)
    features_matrix = embeddings

    # original code
#     features_matrix = np.asarray([model[str(node)] for node in range(len(graph))])

    # use other word2vec lib
#     features_matrix = np.asarray([model[str(node)] for node in range(len(model.vocab)-1)])
#     rand_list = range(len(model.vocab)-1)
# #     random.shuffle(rand_list)
#     features_matrix = np.asarray([model[str(node)] for node in rand_list])
    
    # 2. Shuffle, to create train/test groups
    shuffles = []
    for x in range(1):
        shuffles.append(skshuffle(features_matrix, labels_matrix, random_state = 1))

    # 3. to score each train/test group
    all_results = defaultdict(list)

#     if args.all:
#         training_percents = numpy.asarray(range(1, 10)) * .1
#     else:
#         training_percents = [0.1, 0.5, 0.9]
    training_percents = [0.1]
    for train_percent in training_percents:
        for shuf in shuffles:
            
            X, y = shuf

            training_size = int(train_percent * X.shape[0])

            X_train = X[:training_size, :]
            y_train_ = y[:training_size]

            y_train = [[] for x in range(y_train_.shape[0])]


            cy =  y_train_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_train[i].append(j)

            assert sum(len(l) for l in y_train) == y_train_.nnz

            X_test = X[training_size:, :]
            y_test_ = y[training_size:]

            y_test = [[] for _ in range(y_test_.shape[0])]

            cy =  y_test_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_test[i].append(j)

            clf = TopKRanker(LogisticRegression())
            clf.fit(X_train, y_train_)

            # find out how many labels should be predicted
            top_k_list = [len(l) for l in y_test]
            preds = clf.predict(X_test, top_k_list)

            results = {}
            averages = ["micro", "macro"]
            for average in averages:
                results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

            all_results[train_percent].append(results)

    print ('Results, using embeddings of dimensionality', X.shape[1])
    print ('-------------------')
    for train_percent in sorted(all_results.keys()):
        print ('Train percent:', train_percent)
    for index, result in enumerate(all_results[train_percent]):
        print ('Shuffle #%d:   ' % (index + 1), result)
    avg_score = defaultdict(float)
    for score_dict in all_results[train_percent]:
        for metric, score in iteritems(score_dict):
            avg_score[metric] += score
    for metric in avg_score:
        avg_score[metric] /= len(all_results[train_percent])
    print ('Average score:', dict(avg_score))
    print ('-------------------')
    

# Load Data

In [266]:
# Initialize parameters
DATASET = 'citeseer'

embedding_size = 128
learning_rate = 0.1
gl_learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 10
path_size = 10


use_feature = True
update_emb = True
layer_loss =  True

In [267]:
file_list = ['/hdd2/graph_embedding/customized/tmp/citeseer.embeddings.walks']
# file_list = ['/hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0']
dataset = genfromtxt(file_list[0], delimiter=' ')

def get_num_vacabulary(dataset):
    word_count = 0
    for d in dataset:
        word_count = max(word_count, max(d))
    return int(word_count)

vocabulary_size = get_num_vacabulary(dataset) + 1

In [268]:
words = dataset.flatten()
words = [str(int(w)) for w in words]

In [269]:
words[:10]

['1858', '355', '3029', '583', '2167', '583', '2932', '3029', '3032', '3029']

In [270]:
dataset[0]

array([ 1858.,   355.,  3029.,   583.,  2167.,   583.,  2932.,  3029.,
        3032.,  3029.,  1425.,  3029.,  2101.,  2102.,  2101.,  3029.,
         583.,  2167.,  2164.,  2169.,  3227.,  2721.,  3227.,  2721.,
         774.,  2220.,   774.,  2719.,   774.,  2364.,  2369.,    83.,
        2720.,   774.,  2720.,    83.,   680.,    83.,  1556.,    83.])

In [271]:
def build_dataset(words):
    count = []
#     count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    for word in words:
        if word in dictionary:
            index = dictionary[word]
#         else:
#             index = 0  # dictionary['UNK']
        data.append(index)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

('Most common words (+UNK)', [('2171', 40988), ('3058', 31149), ('2261', 13697), ('906', 13467), ('2319', 12872)])
('Sample data', [260, 160, 16, 763, 2458, 763, 2389, 16, 3116, 16])


In [114]:
# matfile = '/hdd2/graph_embedding/deepwalk/example_graphs/blogcatalog.mat'
# mat = sio.loadmat(matfile)
# A = mat['network']
# graph = sparse2graph(A)
# labels_matrix = mat['group']

In [128]:
# label_list = [row.nonzero()[1][0] for row in labels_matrix]

In [72]:
# def split_dataset(matfile, vocabulary_size, dictionary):
#     # 1. Load labels
#     mat = sio.loadmat(matfile)
#     A = mat['network']
#     graph = sparse2graph(A)
#     labels_matrix = mat['group'].toarray()
    
#     # node id list
#     node_id_list = np.asarray([dictionary[str(node)] for node in range(vocabulary_size)], dtype=np.int32)
    
#     # 2. Shuffle, to create train/test groups
#     shuffles = []
#     for x in range(1):
#         shuffles.append(skshuffle(node_id_list, labels_matrix, random_state = 1))

#     # 3. to score each train/test group
#     all_results = defaultdict(list)

# #     if args.all:
# #         training_percents = numpy.asarray(range(1, 10)) * .1
# #     else:
# #         training_percents = [0.1, 0.5, 0.9]
#     training_percents = [0.9]
#     for train_percent in training_percents:
#         for shuf in shuffles:
            
#             X, y = shuf

#             training_size = int(train_percent * X.shape[0])

#             X_train = X[:training_size]
#             y_train = y[:training_size, :]

#             X_test = X[training_size:]
#             y_test = y[training_size:, :]
        
#     return X_train, y_train, X_test, y_test

# trn_x, trn_y, tst_x, tst_y = split_dataset(matfile, vocabulary_size, dictionary)
# data_splited_filename = '/hdd2/graph_embedding/customized/blogcatalog_splited_p90.pickle'
# data_trn_tst = {'trn_x' : trn_x,
#                 'trn_y' : trn_y,
#                 'tst_x' : tst_x,
#                 'tst_y' : tst_y}
# with open(data_splited_filename, 'wb') as handle:
#     pickle.dump(data_trn_tst, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [269]:
# ## Read the data, the trn_x is the position in the embedding matrix
# with open(data_splited_filename, 'rb') as handle:
#     unserialized_data = pickle.load(handle)
# trn_x, trn_y, tst_x, tst_y = (unserialized_data['trn_x'], unserialized_data['trn_y'], unserialized_data['tst_x'], unserialized_data['tst_y'])

In [272]:
path_index = 0
batch_path_size = 1
batch_size = batch_path_size * 400
def generate_batch(batch_path_size, num_skips, skip_window):
    global path_index
    assert batch_size % num_skips == 0
#     print(num_skips, skip_window)
    assert num_skips <= 2 * skip_window
#     batch = np.ndarray(shape=(batch_size), dtype=np.int32)
#     labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    batch = []
    labels = []
#     span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=batch_path_size)
    for i in range(batch_path_size):
        len_path = len(dataset[path_index])
        for l in range(skip_window, len_path - skip_window): # [ skip_window target skip_window ]
            for m in range(l - skip_window, l + skip_window + 1):
                if m < 0 or m >= len_path or m == l: 
                    continue
                batch.append(dictionary[str(int(dataset[path_index][l]))])
                labels.append(dictionary[str(int(dataset[path_index][m]))])
        
        path_index = (path_index + 1) % len(dataset)
    return (np.asarray(batch, dtype = np.int32), 
            np.asarray(labels, dtype = np.int32).reshape([len(labels), 1]))

# print('data:', [reverse_dictionary[di] for di in data[:8]])

for batch_path_size, num_skips, skip_window in [(1, 20, 10)]:
    data_index = 0
    batch, labels = generate_batch(batch_path_size=batch_path_size, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print(len(batch))
#     print('    batch node id:', [reverse_dictionary[bi] for bi in batch])
#     print('    node position:')
#     print(batch.tolist())
#     print(labels)
#     print('    labels:', [reverse_dictionary[li] for li in labels.reshape(batch_size)])


with num_skips = 20 and skip_window = 10:
400


In [273]:
len(labels)

400

In [274]:
# # inject trained embedding
# model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')
# features_matrix = np.asarray([model[str(node)] for node in range(len(model.vocab)-1)])
# # make the order consistant to the current one
# reordered_list = np.asarray([int(reverse_dictionary[idx]) for idx in range(len(model.vocab)-1)])
# reordered_features_matrix = features_matrix[reordered_list]

In [275]:
# Reproduce Gensim weights initialization
def seeded_vector(seed_string, vector_size):
    """Create one 'random' vector (but deterministic by seed_string)"""
    # Note: built-in hash() may vary by Python version or even (in Py3.x) per launch
    once = np.random.RandomState(hash(seed_string) & 0xffffffff)
    return (once.rand(vector_size) - 0.5) / vector_size

features_list = []
for idx in range(vocabulary_size):
    str_node = reverse_dictionary[idx]
    features_list.append(seeded_vector(str_node + str(1), 128))
features_matrix = np.asarray(features_list)

In [305]:
# for datasets in the icml paper
file_path = '/hdd2/graph_embedding/planetoid/data/'
def get_labeled_instant():
    DATASET = 'citeseer'
    NAMES = ['x', 'y', 'tx', 'ty', 'graph']
    OBJECTS = []
    for i in range(len(NAMES)):
        f = file_path + 'trans.{}.{}'.format(DATASET, NAMES[i])
        print(f)
        OBJECTS.append(cPickle.load(open(f)))
    x, y, tx, ty, graph = tuple(OBJECTS)
    
    trn_idx = np.array([int(dictionary[str(idx)]) for idx in range(x.shape[0])])
#     trn_idx = trn_idx.reshape(trn_idx.shape[0], 1)
    
    tst_idx = np.array([int(dictionary[str(idx)]) for idx in range(x.shape[0], x.shape[0]+tx.shape[0])])
#     tst_idx = tst_idx.reshape(tst_idx.shape[0], 1)
    
    
    return trn_idx, y, tst_idx, ty, x.toarray(), tx.toarray()

trn_idx, trn_y, tst_idx, tst_y, trn_f, tst_f = get_labeled_instant()

/hdd2/graph_embedding/planetoid/data/trans.citeseer.x
/hdd2/graph_embedding/planetoid/data/trans.citeseer.y
/hdd2/graph_embedding/planetoid/data/trans.citeseer.tx
/hdd2/graph_embedding/planetoid/data/trans.citeseer.ty
/hdd2/graph_embedding/planetoid/data/trans.citeseer.graph


In [306]:
trn_idx.shape

(120,)

In [311]:
use_feature = False
use_weight = False
labeled_size = trn_idx.shape[0]
batch_path_size = 2
batch_size = batch_path_size * 400
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 10 # How many words to consider left and right.
num_skips = 20 # How many times to reuse an input to generate a label.
num_class = 6

# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    path_weight = tf.placeholder(tf.float32, shape = [None, None])
        
    # Variables.
#     embeddings = tf.Variable(
#         tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embeddings = tf.Variable(features_matrix, dtype=tf.float32)
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                                   labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

    global_step = tf.Variable(0, trainable=False)
#     learning_rate = tf.train.exponential_decay(
#         learning_rate = 0.25,                # Base learning rate.
#         global_step = global_step,  # Current index into the dataset.
#         decay_steps = 10000,          # Decay step.
#         decay_rate = 0.96,                # Decay rate.
#         staircase=True)

#     emb_lr = tf.train.polynomial_decay(
#         learning_rate = 0.25,                # Base learning rate.
#         global_step = global_step,  # Current index into the dataset.
#         decay_steps = 10000,          # Decay step.
#         end_learning_rate = 0.025,
#         power = 1)
    
    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.25).minimize(loss, global_step=global_step)

    
#     clf_lr = 0.25
    clf_idx = tf.placeholder(tf.int32, shape=[None])
    clf_y = tf.placeholder(tf.float32, shape=[None, trn_y.shape[1]])
    
    
    embed_x = tf.nn.embedding_lookup(embeddings, clf_idx)

#   for datasets in the deepwalk, multi-class
#     logit_y = tf.layers.dense(inputs = embed_x, units = clf_y.shape[1], 
#                               activation=tf.nn.sigmoid, kernel_initializer=glorot_uniform_initializer())
#     clf_loss = tf.reduce_mean(
#         tf.nn.sigmoid_cross_entropy_with_logits(logits = logit_y, labels = clf_y))

    feature_dataset = tf.placeholder(tf.float32, shape=[None, trn_f.shape[1]])
    l_x_hid = tf.layers.dense(inputs = feature_dataset, units = clf_y.shape[1],
                              activation = tf.nn.softmax, kernel_initializer = glorot_uniform_initializer())
    if (use_feature):
        logit_emd = tf.layers.dense(inputs = embed_x, units = clf_y.shape[1],
                                    activation=tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
        l_f = tf.concat([l_x_hid, logit_emd], axis = 1)
        logit_y = tf.layers.dense(inputs = l_f, units = clf_y.shape[1],
                                  activation=tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
    else:
    #   for datasets in the icml paper, single-class
        logit_y = tf.layers.dense(inputs = embed_x, units = clf_y.shape[1], 
                                  activation=tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
        
    clf_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logit_y, labels = clf_y))
    
    clf_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.25).minimize(clf_loss)
    
    
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
#     norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
#     normalized_embeddings = embeddings / norm
#     valid_embeddings = tf.nn.embedding_lookup(
#         normalized_embeddings, valid_dataset)
#     similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [290]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    batch_data, batch_labels = generate_batch(
        batch_path_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l, res_embed = session.run([optimizer, loss, embed], feed_dict=feed_dict)
    print('loss: %.4f' %(l))

loss: 4.0610


In [298]:
reverse_dictionary[1025]

'6156'

In [299]:
dictionary['6156']

1025

In [305]:
print(labels_matrix[6156])

  (0, 2)	1.0
  (0, 4)	1.0
  (0, 12)	1.0
  (0, 16)	1.0
  (0, 23)	1.0
  (0, 24)	1.0


In [310]:
use_feature = False
emb_steps = 10000 #50000001
clf_steps = 1000

total_step = 0
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    while (True):
        average_emb_loss = 0
        average_clf_loss = 0
        for step in range(emb_steps):
            batch_data, batch_labels = generate_batch(
                batch_path_size=batch_path_size, num_skips=num_skips, skip_window=skip_window)
            feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
            _, l = session.run([optimizer, loss], feed_dict=feed_dict)
            average_emb_loss += l
            if step % 2000 == 0:
                if step > 0:
                    average_emb_loss = average_emb_loss / 2000.0
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average embedding loss at step %d: %f, lr %f' % (step, average_emb_loss, 0.25))
                average_emb_loss = 0
                
        for step in range(clf_steps):
            # for datasets in the icml paper
            feed_dict = {clf_idx : trn_idx, clf_y : trn_y, feature_dataset : trn_f}
            _, l = session.run([clf_optimizer, clf_loss], feed_dict=feed_dict)
            average_clf_loss += l
            if step % 1000 == 0:
                if step > 0:
                    average_clf_loss = average_clf_loss / 1000.0
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average classification loss at step %d: %f, lr %f' % (step, average_clf_loss, 0.25))
                average_clf_loss = 0
                
        # note that this is expensive (~20% slowdown if computed every 500 steps)
#         y_p = tf.argmax(logit_y, 1)
#         feed_dict = {clf_x : tst_x, clf_y : tst_y}
#         _, l, res_logit_y = session.run([clf_optimizer, clf_loss, logit_y], feed_dict=feed_dict)
#         y_true = np.argmax(tst_y,1)
#         print("micro: ", f1_score(y_true, res_pred_y, average='micro'))
#         print("macro: ", f1_score(y_true, res_pred_y, average='macro'))

        res_y_pred = tf.argmax(logit_y, 1)
        res_y_true = tf.argmax(clf_y, 1)
#         correct_prediction = tf.equal(tf.argmax(logit_y), tf.argmax(clf_y))
#         accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#         print("Epoch %s:" % total_step, 
#               accuracy.eval({clf_x: trn_x, clf_y: trn_y}),
#               accuracy.eval({clf_x: tst_x, clf_y: tst_y}))
        
        trn_y_pred = res_y_pred.eval({clf_idx : trn_idx, clf_y: trn_y, feature_dataset : trn_f})
        trn_y_ture = res_y_true.eval({clf_idx : trn_idx, clf_y: trn_y, feature_dataset : trn_f})
        tst_y_pred = res_y_pred.eval({clf_idx : tst_idx, clf_y: tst_y, feature_dataset : tst_f})
        tst_y_ture = res_y_true.eval({clf_idx : tst_idx, clf_y: tst_y, feature_dataset : tst_f})
        
        print("Epoch %d, trn acc %.6f acc %.6f:" % (total_step,
                                                  accuracy_score(trn_y_ture, trn_y_pred.flatten()),
                                                  accuracy_score(tst_y_ture, tst_y_pred.flatten())))
        
        if total_step % 1 == 0:
            embedding_filename = '/hdd2/graph_embedding/customized/results/exp_citeseer_semi/citeseer_embeddings_iter%d.txt' %total_step
            not_normal_embeddings = embeddings.eval()
            ordered_embeddings = [not_normal_embeddings[dictionary[str(node)]] for node in range(len(dictionary))]
            np.savetxt(embedding_filename, ordered_embeddings)
        
        total_step += 1
            

    final_embeddings = normalized_embeddings.eval()
    not_normal_embeddings = embeddings.eval()

Initialized
Average embedding loss at step 0: 3.519383, lr 0.250000
Average embedding loss at step 2000: 3.926005, lr 0.250000
Average embedding loss at step 4000: 3.765736, lr 0.250000
Average embedding loss at step 6000: 3.484224, lr 0.250000
Average embedding loss at step 8000: 2.931002, lr 0.250000
Average classification loss at step 0: 1.791780, lr 0.250000
Epoch 0, trn acc 0.666667 acc 0.525000:
Average embedding loss at step 0: 0.320834, lr 0.250000
Average embedding loss at step 2000: 1.678072, lr 0.250000
Average embedding loss at step 4000: 1.363678, lr 0.250000
Average embedding loss at step 6000: 1.142598, lr 0.250000
Average embedding loss at step 8000: 1.006780, lr 0.250000
Average classification loss at step 0: 1.477347, lr 0.250000
Epoch 1, trn acc 1.000000 acc 0.499000:
Average embedding loss at step 0: 1.097978, lr 0.250000
Average embedding loss at step 2000: 0.850220, lr 0.250000
Average embedding loss at step 4000: 0.842479, lr 0.250000
Average embedding loss at st

KeyboardInterrupt: 

In [308]:
logit_y.shape

TensorShape([Dimension(None), Dimension(6)])

In [242]:
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings_1.txt'
ordered_embeddings = [final_embeddings[dictionary[str(node)]] for node in range(len(dictionary))]
np.savetxt(embedding_filename, ordered_embeddings)

NameError: name 'final_embeddings' is not defined

In [227]:
res_logit_y[0]

array([ 0.47477195,  0.47871515,  0.47899124,  0.4759534 ,  0.47790405,
        0.47898862,  0.47662544,  0.48324525,  0.47535485,  0.47714397,
        0.47675049,  0.47513527,  0.47733408,  0.4759779 ,  0.47573614,
        0.47668341,  0.47703272,  0.47557175,  0.47846189,  0.47588319,
        0.47464845,  0.47754869,  0.47674361,  0.48117602,  0.47582838,
        0.47704467,  0.47620195,  0.47587633,  0.47659436,  0.47654676,
        0.47580355,  0.47573981,  0.47579864,  0.47626203,  0.47682789,
        0.47790003,  0.47652382,  0.47664452,  0.47489908], dtype=float32)

In [221]:
tst_y

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [259]:
embeddings = np.loadtxt(embedding_filename)

In [260]:
embeddings[1209]

array([ 58.47768402, -14.06970406, -11.84331131,   3.77727461,
        27.18597221, -25.0004673 ,  65.72541809, -65.32319641,
        46.84730148, -17.94046402,  -9.88634205,   4.96879911,
       -39.52791977, -41.69575119,  74.56922913,   7.61631155,
        23.03376389,  50.97651291, -33.59208679,  31.2266407 ,
        81.93914032,  15.72993279,  -5.28370953,  17.18107605,
       -75.29068756,  14.2514925 ,   7.94441748,  -4.89663982,
       -51.30142212, -33.08482361, -32.42542267,   1.93974102,
        46.80765533,   8.50759125,  10.44501019, -40.15472794,
        -5.02595329, -36.50287247, -45.88523102, -79.72190094,
       -17.26217461,   8.80545712, -52.35463333, -16.01053047,
       -43.30438232, -39.79665756, -16.72794151,  34.34871674,
        57.16449738,  -4.06061411, -72.57341003,  -7.9510541 ,
        63.76834869,  43.05852509,   2.076653  , -98.20155334,
        16.07836723,  46.57059479,  68.2883606 ,  -1.08950758,
         5.00839376,   9.26826859,   8.00768375,  18.40

In [150]:
np.unique(dictionary.values())

array([    0,     1,     2, ..., 10308, 10309, 10310])

In [152]:
np.unique(dataset.flatten())

array([  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
         1.03090000e+04,   1.03100000e+04,   1.03110000e+04])

In [153]:
vocabulary_size

10312

In [155]:
len(reverse_dictionary)

10311

In [270]:
from keras.preprocessing import sequence
from keras.preprocessing.sequence import skipgrams

In [273]:
sampling_table = sequence.make_sampling_table(vocabulary_size)
couples, labels = skipgrams(data, vocabulary_size, window_size=10, sampling_table=sampling_table, negative_samples=5)
# word_target, word_context = zip(*couples)
# word_target = np.array(word_target, dtype="int32")
# word_context = np.array(word_context, dtype="int32")

# print(couples[:10], labels[:10])

KeyboardInterrupt: 

In [26]:
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')

target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

In [36]:
i = 0
for line in walks_corpus:
    i += 1
print(i)

824960


In [21]:
# model = Skipgram(sentences=walks_corpus, vocabulary_counts=vocabulary_size, size=128,
#                  window=10, min_count=0, trim_rule=None, workers=1)

In [31]:
1

1

In [89]:
def sparse2graph(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k,v in iteritems(G)}

In [90]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [134]:
def scoring(emb_filename, matfile):
    # 0. Files
    embeddings_file = emb_filename

    # 1. Load Embeddings
#     embeddings = np.loadtxt(embeddings_file)
    
    ## for original deepwalk
    #model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)
    
    ## for external word2vec lib
    model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

    # 2. Load labels
    mat = sio.loadmat(matfile)
    A = mat['network']
    graph = sparse2graph(A)
    labels_matrix = mat['group']
    labels_count = labels_matrix.shape[1]
    mlb = MultiLabelBinarizer(range(labels_count))

    # Map nodes to their features (note:  assumes nodes are labeled as integers 1:N)
#     features_matrix = embeddings

#     features_matrix = np.asarray([model[str(node)] for node in range(len(graph))])

    # use other word2vec lib
    features_matrix = np.asarray([model[str(node)] for node in xrange(len(model.vocab)-1)])
    
    # 2. Shuffle, to create train/test groups
    shuffles = []
    for x in range(1):
        shuffles.append(skshuffle(features_matrix, labels_matrix))

    # 3. to score each train/test group
    all_results = defaultdict(list)

#     if args.all:
#         training_percents = numpy.asarray(range(1, 10)) * .1
#     else:
#         training_percents = [0.1, 0.5, 0.9]
    training_percents = [0.1]
    for train_percent in training_percents:
        for shuf in shuffles:
            
            X, y = shuf

            training_size = int(train_percent * X.shape[0])

            X_train = X[:training_size, :]
            y_train_ = y[:training_size]

            y_train = [[] for x in range(y_train_.shape[0])]


            cy =  y_train_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_train[i].append(j)

            assert sum(len(l) for l in y_train) == y_train_.nnz

            X_test = X[training_size:, :]
            y_test_ = y[training_size:]

            y_test = [[] for _ in range(y_test_.shape[0])]

            cy =  y_test_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_test[i].append(j)

            clf = TopKRanker(LogisticRegression())
            clf.fit(X_train, y_train_)

            # find out how many labels should be predicted
            top_k_list = [len(l) for l in y_test]
            preds = clf.predict(X_test, top_k_list)

            results = {}
            averages = ["micro", "macro"]
            for average in averages:
                results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

            all_results[train_percent].append(results)

    print ('Results, using embeddings of dimensionality', X.shape[1])
    print ('-------------------')
    for train_percent in sorted(all_results.keys()):
        print ('Train percent:', train_percent)
    for index, result in enumerate(all_results[train_percent]):
        print ('Shuffle #%d:   ' % (index + 1), result)
    avg_score = defaultdict(float)
    for score_dict in all_results[train_percent]:
        for metric, score in iteritems(score_dict):
            avg_score[metric] += score
    for metric in avg_score:
        avg_score[metric] /= len(all_results[train_percent])
    print ('Average score:', dict(avg_score))
    print ('-------------------')

In [92]:
matfile = '/hdd2/graph_embedding/deepwalk/example_graphs/blogcatalog.mat'
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
embedding_filename_other_lib = '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin'

scoring('', matfile)

NameError: global name 'word2vec' is not defined

In [23]:
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
model = KeyedVectors.load_word2vec_format(embedding_filename_original, binary=False)
                                          
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embeddings = np.loadtxt(embedding_filename)


In [22]:
model['0']

array([ 0.110529  ,  0.19354101,  0.3969    ,  0.16305301, -0.198276  ,
       -0.219273  ,  0.045498  ,  0.138824  , -0.174495  ,  0.20618799,
        0.48984301,  0.302856  , -0.46013999,  0.227097  ,  0.071323  ,
       -0.43010399,  0.015921  , -0.021136  ,  0.28175899, -0.26225701,
        0.102745  , -0.31963301,  0.058964  ,  0.132149  ,  0.176946  ,
       -0.155433  ,  0.160973  , -0.282166  ,  0.030017  , -0.165079  ,
        0.25168899, -0.52189702,  0.005448  ,  0.18592501, -0.013992  ,
       -0.070675  ,  0.033961  ,  0.117675  , -0.073083  ,  0.068748  ,
       -0.10755   ,  0.068676  ,  0.17162   , -0.136898  ,  0.17979699,
       -0.106551  , -0.212037  ,  0.103523  , -0.242975  , -0.46731299,
        0.182107  ,  0.092075  , -0.141946  ,  0.051342  ,  0.31653801,
        0.28085399, -0.029812  ,  0.19881   ,  0.31846499, -0.12293   ,
        0.22415499, -0.21315201, -0.220193  , -0.15813901,  0.104372  ,
       -0.038969  ,  0.020534  , -0.18385699, -0.049377  ,  0.24

In [26]:
embeddings[0]

array([ -1.47322476e-01,   2.67052501e-02,   1.52672976e-02,
        -1.37635857e-01,  -6.47586957e-02,  -2.46349778e-02,
        -8.18009302e-02,   6.91710338e-02,  -9.66271833e-02,
        -9.89247933e-02,   5.70955575e-02,   1.53366596e-01,
         8.30798447e-02,  -1.44452751e-01,   9.11711424e-04,
         1.01996966e-01,   4.97803837e-02,  -4.91250493e-02,
         1.03872299e-01,  -1.11238003e-01,   3.35200056e-02,
         6.37169033e-02,  -3.92291024e-02,  -2.92820130e-02,
        -2.85616820e-03,  -3.53677608e-02,  -5.77375665e-02,
        -3.55093554e-02,   4.43368331e-02,  -1.63600937e-01,
        -5.06901741e-02,   1.09737791e-01,  -6.03695633e-03,
        -9.09792027e-04,  -9.44678709e-02,  -5.35581484e-02,
         5.67527264e-02,  -2.60556079e-02,   1.89456977e-02,
         2.38609854e-02,   4.84210551e-02,   1.34210438e-01,
        -1.30895942e-01,   6.51589632e-02,   6.92558140e-02,
        -5.88307390e-03,  -1.87402926e-02,   1.52439341e-01,
        -7.65966475e-02,

In [7]:
mat = sio.loadmat(matfile)
A = mat['network']
labels_matrix = mat['group']
labels_count = labels_matrix.shape[1]
mlb = MultiLabelBinarizer(range(labels_count))

TypeError: 'MultiLabelBinarizer' object is not iterable

In [27]:
# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [28]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-03-13 21:07:11,853 : INFO : collecting all words and their counts
2018-03-13 21:07:11,855 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-13 21:07:11,857 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2018-03-13 21:07:11,858 : INFO : Loading a fresh vocabulary
2018-03-13 21:07:11,860 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-03-13 21:07:11,863 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-03-13 21:07:11,865 : INFO : deleting the raw counts dictionary of 3 items
2018-03-13 21:07:11,868 : INFO : sample=0.001 downsamples 3 most-common words
2018-03-13 21:07:11,870 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-03-13 21:07:11,872 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-03-13 21:07:11,874 : INFO : resetting layer weights
2018-03-13 21:07:11,876 : INFO : training model with 3 workers o

In [87]:
init_iter_label = 10
init_iter_graph = 7
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

In [17]:
max_iter = 1
iter_graph = 0
iter_inst = 5
iter_label = 0
use_reweight = False

In [92]:
# with tf.Session() as session:
#     gx, gy = next(label_generator)
#     tf.global_variables_initializer().run()
#     feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#     _, loss = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#     print 'iter label', i, loss

In [93]:
res_reweight[0]

array([ 0.00497301], dtype=float32)

In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

In [250]:
import word2vec

In [15]:
word2vec.word2phrase('/hdd2/graph_embedding/tmp/text8', '/hdd2/graph_embedding/tmp/text8-phrases', verbose=True)

Starting training using file /hdd2/graph_embedding/tmp/text8
Words processed: 17000K     Vocab size: 4399K  
Vocab size (unigrams + bigrams): 2419827
Words in train file: 17005206


In [16]:
word2vec.word2vec('/hdd2/graph_embedding/tmp/text8-phrases', '/hdd2/graph_embedding/tmp/text8.bin', size=100, verbose=True)

Starting training using file /hdd2/graph_embedding/tmp/text8-phrases
Vocab size: 98331
Words in train file: 15857306
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 467.25k  

In [251]:
word2vec.word2vec(data,
                  '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.1.bin', 
                  size=128, verbose=True)

OSError: [Errno 7] Argument list too long

In [22]:
model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

In [40]:
model.vocab

array([u'</s>', u'4838', u'175', ..., u'9756', u'1678', u'2931'], 
      dtype='<U78')

In [52]:
features_matrix = np.asarray([model[str(node)] for node in xrange(len(model.vocab)-1)])

In [48]:
model.vocab

array([u'</s>', u'4838', u'175', ..., u'9756', u'1678', u'2931'], 
      dtype='<U78')

In [38]:
model.vectors[1]

array([ 0.03300432, -0.07565996, -0.16463131, -0.10145742,  0.00168844,
       -0.09007662, -0.03259023, -0.22714335, -0.00282227, -0.05083675,
       -0.07032426, -0.15832779, -0.13998848,  0.07574239,  0.01739285,
        0.05170207, -0.01704509, -0.04309488,  0.13984987, -0.07098495,
       -0.08131668, -0.06108802,  0.03238546,  0.03199002, -0.14047897,
        0.1192169 , -0.11983237,  0.17141342, -0.04238614, -0.07682881,
       -0.09226497, -0.08641661, -0.05119943,  0.04880232,  0.05202007,
        0.11911941,  0.02494194,  0.0460702 , -0.02544389,  0.02227429,
       -0.02918422,  0.08963519, -0.01449389,  0.13147458,  0.04316   ,
        0.05062373,  0.01030556,  0.14064927,  0.05300813,  0.09054667,
        0.16458763,  0.00496632,  0.09853401, -0.09157445,  0.03466847,
       -0.192109  , -0.04286476,  0.0337339 ,  0.09414463,  0.02883872,
       -0.12935385,  0.05580147,  0.04444493,  0.12305645, -0.06843153,
       -0.05650309, -0.06930542, -0.0933516 , -0.08945961,  0.03

In [50]:
model['0']

array([-0.03359349, -0.01812873, -0.00173358,  0.01964512,  0.10055595,
        0.02292307,  0.0607472 ,  0.04138128, -0.12003173, -0.09744611,
        0.07093195, -0.01193107,  0.07130294,  0.08676811,  0.11344255,
        0.05344105, -0.02284033,  0.05260049, -0.01932774, -0.09711266,
       -0.19951601,  0.00738851, -0.07712144, -0.07025649, -0.03221126,
       -0.13667656,  0.08364721, -0.04125704, -0.03211425,  0.06983157,
       -0.00799658, -0.01177289, -0.16706869,  0.07463736, -0.09091619,
       -0.11201481,  0.08337475,  0.05930549, -0.07940312, -0.05631904,
       -0.01991395, -0.02349721,  0.1824768 ,  0.01132163, -0.04610476,
        0.01006832,  0.04507252, -0.08544502,  0.02825145,  0.02216966,
        0.03565323, -0.06984831,  0.03623525,  0.1129318 , -0.09319748,
       -0.04915803,  0.11094059,  0.0029751 , -0.08399934,  0.21738739,
        0.06743176, -0.09108514,  0.01798326, -0.08503798,  0.0290214 ,
        0.07596923, -0.02369601,  0.11986063, -0.04430954, -0.07

In [53]:
features_matrix[0]

array([-0.03359349, -0.01812873, -0.00173358,  0.01964512,  0.10055595,
        0.02292307,  0.0607472 ,  0.04138128, -0.12003173, -0.09744611,
        0.07093195, -0.01193107,  0.07130294,  0.08676811,  0.11344255,
        0.05344105, -0.02284033,  0.05260049, -0.01932774, -0.09711266,
       -0.19951601,  0.00738851, -0.07712144, -0.07025649, -0.03221126,
       -0.13667656,  0.08364721, -0.04125704, -0.03211425,  0.06983157,
       -0.00799658, -0.01177289, -0.16706869,  0.07463736, -0.09091619,
       -0.11201481,  0.08337475,  0.05930549, -0.07940312, -0.05631904,
       -0.01991395, -0.02349721,  0.1824768 ,  0.01132163, -0.04610476,
        0.01006832,  0.04507252, -0.08544502,  0.02825145,  0.02216966,
        0.03565323, -0.06984831,  0.03623525,  0.1129318 , -0.09319748,
       -0.04915803,  0.11094059,  0.0029751 , -0.08399934,  0.21738739,
        0.06743176, -0.09108514,  0.01798326, -0.08503798,  0.0290214 ,
        0.07596923, -0.02369601,  0.11986063, -0.04430954, -0.07